In [ ]:
# 딥러닝 CNN기법으로 적용한 모델
# 결과값 출력이 다음과 같이 나옴
#✅ 학습 시작
#[Epoch 1] Loss: 34.5591 | Val Accuracy: 89.26%
#[Epoch 2] Loss: 25.4763 | Val Accuracy: 90.88%
#[Epoch 3] Loss: 22.5501 | Val Accuracy: 90.94%
#[Epoch 4] Loss: 19.6723 | Val Accuracy: 91.90%
#[Epoch 5] Loss: 17.4573 | Val Accuracy: 93.25%
#[Epoch 6] Loss: 15.0400 | Val Accuracy: 93.49%
#[Epoch 7] Loss: 12.1350 | Val Accuracy: 93.16%
#[Epoch 8] Loss: 9.8619 | Val Accuracy: 94.15%
#[Epoch 9] Loss: 7.9755 | Val Accuracy: 93.55%
#[Epoch 10] Loss: 6.6986 | Val Accuracy: 93.16%
#🧪 테스트 정확도: 93.34%
# 꽤 높은 정확도, 교육에서 배운 epoch를 조절해서 교육하는 방식으로 이용할 수 있음



import os
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset

# -------------------------
# [1] 설정
# -------------------------
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001
IMG_SIZE = 90
MAX_PER_CLASS = 1666  # 클래스당 샘플 수

train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir   = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir  = r"C:\Users\Users\open-closed-eyes-dataset\test"

# -------------------------
# [2] 전처리 정의
# -------------------------
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# -------------------------
# [3] 클래스별 균형 샘플링 함수
# -------------------------
def balanced_subset(dataset, max_per_class):
    targets = np.array(dataset.targets)
    indices = []

    for class_idx in range(len(dataset.classes)):
        class_indices = np.where(targets == class_idx)[0]
        sampled = random.sample(list(class_indices), min(len(class_indices), max_per_class))
        indices.extend(sampled)

    return Subset(dataset, indices)

# -------------------------
# [4] 데이터셋 및 로더 생성
# -------------------------
train_dataset = balanced_subset(ImageFolder(train_dir, transform=transform), MAX_PER_CLASS)
val_dataset   = balanced_subset(ImageFolder(val_dir, transform=transform), MAX_PER_CLASS)
test_dataset  = balanced_subset(ImageFolder(test_dir, transform=transform), MAX_PER_CLASS)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -------------------------
# [5] CNN 모델 정의
# -------------------------
class EyeCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),  # 90x90 -> 90x90
            nn.ReLU(),
            nn.MaxPool2d(2),                 # 90x90 -> 45x45

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                 # 45x45 -> 22x22

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                 # 22x22 -> 11x11
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 11 * 11, 128),
            nn.ReLU(),
            nn.Linear(128, 2)  # 2 classes: open/closed
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# -------------------------
# [6] 학습 설정
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EyeCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# -------------------------
# [7] 학습 루프
# -------------------------
print("✅ 학습 시작")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 검증 정확도
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs).argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += len(labels)

    val_acc = correct / total * 100
    print(f"[Epoch {epoch+1}] Loss: {running_loss:.4f} | Val Accuracy: {val_acc:.2f}%")

# -------------------------
# [8] 테스트 정확도
# -------------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += len(labels)

test_acc = correct / total * 100
print(f"🧪 테스트 정확도: {test_acc:.2f}%")